In [ ]:
# EXAMPLE 2 - Create linear regression models for all 60 pole transformers.
# Read PI data via PI Web API.

# Import Python packages.
import adodbapi as ado                        # Support for accessing MS SQL to store model coefficients.
import pandas as pd                           # Dataframe support.

from sklearn.linear_model import LinearRegression # Linear regression model from the scikit-learn package.

# Import packages
import json, requests, urllib.parse, requests_kerberos # Support access to PI Web API.

# Setup Kerberos cnnection.
from requests_kerberos import HTTPKerberosAuth, OPTIONAL
clientAuth = HTTPKerberosAuth(force_preemptive=True, mutual_authentication=OPTIONAL, delegate=True)

# Disable warning messages.
requests.packages.urllib3.disable_warnings()

# Open session.
session = requests.session()

In [ ]:
# Connect to PI System at highest level.
print('\nSending top level PI Web API Request\n')
    
# Base URL to PI System.
baseurl = "https://PISRV01/piwebapi/"

# Post URL to PI System and retrieve response.
response = session.get(baseurl, auth=clientAuth, verify=False,  timeout=30)
    
# Response status - did it work, i.e. 200 means success?
print('Response Received: {0} ' .format(response.status_code),'\n')
    
# Unpack items of interest using the "json" package.  Extracts dictionary of links.
Links = json.loads(response.text)['Links']

# Extracts base level URL's for PI Web API from the Links dictionary using the "json" package.
print('Self:', Links['Self'])
print('Asset Servers:', Links['AssetServers'])
print('Data Servers:', Links['DataServers'])
print('PI System:', Links['System'], '\n') 

In [ ]:
# Get WebId for circuit element, "Colegio Clientifico".

# Root path to "circuit" level in AF heirarchy.
circuitPath = '\\\\pisrv01\\Distribution Network\\Alajuela\\Avenida Central\\Transformer 1\\Colegio Científico'

# Construct URL to the Get Elements controller of the PI Web API.
baseurl_elements = baseurl + '/elements'

# Set parameters list by specifying the AF path to the circuit's element.
parameters = {'path': circuitPath,}

# Format parameter list for URL.
url_parameters = urllib.parse.urlencode(parameters, quote_via=urllib.parse.quote)

# Show HTTPS request string.
print('PI Web API Request for WebId of Colegio Científico. Click it to see the response from PI.\n\n', \
      baseurl_elements + '?' + url_parameters, '\n')

# Post URL with path and paramters to PI System
response = session.get(baseurl_elements, auth=clientAuth, params=url_parameters, verify=False, timeout=30)

# Response status - did it work?
print('Response Received: {0} ' .format(response.status_code),'\n')

# Show element's Web_Id.
circuitWebId = json.loads(response.text)['WebId']
print("Colegio Clientifico element's WebId:\n", circuitWebId)

In [ ]:
# Get names and WebId's for phase elements under the "Colegio Científico" circuit element.

# Define dictionary to store the names and WebId's for the three phase elements
phaseElements = {}

# Construct URL to all child elements of circuit "Colegio Científico" using it's WebId.
baseurl_elements = baseurl + '/elements/' + circuitWebId + '/elements'

# Request names and WebId's for all child elements derived from the "Phase" template.
parameters = {'templateName': 'Phase', 'selectedFields':'Items.Name;Items.WebId'}

# Format parameter list for URL.
url_parameters = urllib.parse.urlencode(parameters, quote_via=urllib.parse.quote)

# Show HTTPS request string.
print("PI Web API Request for WebId's of X, Y and Z Phase elements.", \
      "Click it to see the response from PI.\n\n", baseurl_elements + '?' + url_parameters, '\n')

# Post URL with path and paramters to PI System
response = session.get(baseurl_elements, auth=clientAuth, params=url_parameters, verify=False, timeout=30)

# Response status - did it work?
print('Response Received: {0} ' .format(response.status_code),'\n')

#  Loop throught he response to pupulate the phaseElements dictinary.
for item in json.loads(response.text)['Items']:

    # Store the strings.
    phaseElements.update({item['Name']:item['WebId']})
    
# Print the WebIds using the Name index.
print("X Phase's WebId:\n", phaseElements['X Phase'])
print("Y Phase's WebId:\n", phaseElements['Y Phase'])
print("Z Phase's WebId:\n", phaseElements['Z Phase'])   

In [ ]:
# Create dataframe of interpolated data for each transformer.

# Create empty pandas dataframe for transformer data.
columns =['Transformer', 'TimeStamp', 'Temperature', 'Humidity','Wind','Wh Load', 'Wh Load-14d', 'Wh Load-7d']
modellingData = pd.DataFrame(columns=columns)

# As we loop through heirarchy to get data, establish a list of all transformers.
allTransformers = []

In [ ]:
# Fill dataframe by stepping through the phase level and appending each transformer's attribute values. 

# Print URL as example on the first time through the loop.
printURL = True

# Step through phases.
for phaseName, phaseWebId in phaseElements.items():
    
    # Constructed URL to reference each phase element.
    baseurl_referencedelements = baseurl + '/elements' + '/' + phaseWebId + '/referencedelements'

    # Request for all child elements derived from the "Single Phase Transfrmer" template.
    parameters = {'templateName': 'Single Phase Transformer','selectedFields':'Items.Name;Items.WebId'}

    # Format parameter list for URL.
    url_parameters = urllib.parse.urlencode(parameters, quote_via=urllib.parse.quote)

    # Show HTTPS request string
    if printURL:
        print("Sample PI Web API request for Name and WebId's of transformers under",\
            phaseName, ' Click it to see the response from PI.\n\n', \
            baseurl_referencedelements + '?' + url_parameters, '\n')

    # Post URL with path and paramters to PI System
    response = session.get(baseurl_referencedelements, auth=clientAuth, params=url_parameters, verify=False,  timeout=30)

    # Response status - did it work?
    print('Response Received: {0}' .format(response.status_code),"Transformer Name and WebID's for", phaseName, '\n')
    
    # Define\empty dictionary to hold transformer names and WebId's.
    transformerElements = {}

    # Loop through list and put names and WebId's into dictinary.
    for item in json.loads(response.text)['Items']:

        # Store the strings.
        transformerElements.update({item['Name']:item['WebId']})
        
    # Loop through phases.
    for transformerName, transformerWebId in transformerElements.items():
        
        # Add this transformer to the list.
        allTransformers.append(transformerName)

        # Construct URL for getting selected attributes.
        baseurl_multipleattributes = baseurl + '/attributes/multiple'

        # Request WebId's for specified transformer attributes.
        parameters = [('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Ambient Temperature'), \
                      ('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Relative Humidity'), \
                      ('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Wh Delivered Load'), \
                      ('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Wh Delivered Load - 14d'), \
                      ('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Wh Delivered Load - 7d'), \
                      ('Path', circuitPath + '\\' + phaseName + '\\' + transformerName + '|Wind Speed'), \
                      ('SelectedFields','Items.Object.Name;Items.Object.WebId')]

        # Format parameter list for URL.
        url_parameters = urllib.parse.urlencode(parameters, quote_via=urllib.parse.quote)

        # Show HTTPS request string
        if printURL:
            print("Sample PI Web API request for Name an WebId's of", transformerName, "'s selected attributes.",\
                ' Click it to see the response from PI.\n\n', baseurl_multipleattributes + '?' + url_parameters, '\n')

        # Post URL with path and paramters to PI System.
        response = session.get(baseurl_multipleattributes, auth=clientAuth, params=url_parameters, verify=False,  timeout=30)

        # Response status - did it work?
        print('     Response Received: {0}' .format(response.status_code),"Attribute Name and WebId's for", \
              transformerName, '\n')
    
        # Define list for attribute Webid's.
        attributeWebId = []

        # Loop through unnamed list and extract WebId and Name into lists.
        for attributes in json.loads(response.text)['Items']:

            attribute = attributes['Object']
            attributeWebId.append(attribute['WebId'])

        # Construct URL for getting interpoated data for attributes.
        baseurl_streamsets = baseurl + '/streamsets/interpolated'

        # Request for start and end times, and interval.
        parameters = [('WebId', attributeWebId[0]),('WebId', attributeWebId[1]),\
                      ('WebId', attributeWebId[2]),('WebId', attributeWebId[3]),\
                      ('WebId', attributeWebId[4]),('WebId', attributeWebId[5]),\
                      ('startTime','15-jun-2017'),('endTime','31-aug-2017'),('interval','1h'),\
                      ('selectedFields', 'Items.Name;Items.Items.Timestamp;Items.Items.Value')]

        # Format parameter list for URL.
        url_parameters = urllib.parse.urlencode(parameters, quote_via=urllib.parse.quote)

        # Show HTTPS request string
        if printURL:
            print("Sample PI Web API request for Timestamp and Values for transformer's selected attributes.",\
                ' Click it to see the response from PI.\n\n', baseurl_streamsets + '?' + url_parameters, '\n')

        # Post URL with path and paramters to PI System
        response = session.get(baseurl_streamsets, auth=clientAuth, params=url_parameters, verify=False,  timeout=30)

        # Response status - did it work?
        print('     Response Received: {0}' .format(response.status_code), "Attribute Timestamp and Values for", \
              transformerName, '\n')
        
        # Initialize lists to store unpacked data.
        transformer = []; timestamp = []; temperature = []; humidity= []; wind = []
        load = []; load_14d = []; load_7d = []

        # Loop through response and create pandas dataframe for this transformer.
        # For each attribute.
        for attribute in json.loads(response.text)['Items']:

            # For each teimstamp\value.
            for value in attribute['Items']:

                # Store values for "Ambient Temperature" attribute into its list.
                if attribute['Name'] == 'Ambient Temperature':
                    temperature.append(value['Value'])

                    # Fill timestamp and transformer name lists here, only need to do this once.
                    timestamp.append(value['Timestamp'])
                    transformer.append(transformerName)

                # Store values for "Relative Humidity" attribute into its list.  
                if attribute['Name'] == 'Relative Humidity':
                    humidity.append(value['Value'])

                # Store values for "Wind Speed" attribute into its list.  
                if attribute['Name'] == 'Wind Speed':
                    wind.append(value['Value'])

                # Store values for "Wh Delivered Load" attribute into its list.  
                if attribute['Name'] == 'Wh Delivered Load':
                    load.append(value['Value'])

                # Store values for "Wh Delivered Load - 14d" attribute into its list.  
                if attribute['Name'] == 'Wh Delivered Load - 14d':
                    load_14d.append(value['Value'])

                # Store values for "Wh Delivered Load - 7d" attribute into its list.  
                if attribute['Name'] == 'Wh Delivered Load - 7d':
                    load_7d.append(value['Value'])

        # Put lists of data for this transformer into a temporary dataframe
        thisTransformerData = pd.DataFrame({'Transformer':transformer, 'TimeStamp':timestamp, 'Temperature':temperature, \
                                            'Humidity':humidity, 'Wind':wind, 'Wh Load':load, 'Wh Load-14d':load_14d, \
                                            'Wh Load-7d':load_7d})

        # Add this transformer's data to the others.
        modellingData = pd.concat([modellingData, thisTransformerData],ignore_index=True)

        # Turn off URL printing.
        printURL = False

# Print end of dataframe to see if we got all 110,940 records(row index of 110,939).
modellingData.tail(5)

In [ ]:
# Using the "adodbapi" package, to connect to the "Predictive Equations" MS SQL.

# In order to analyze transformers individually, we need to set the datframe's index to the "Transformer" column. 
modellingData = modellingData.set_index("Transformer", drop=False)

# Set connection parameters.
con_string = 'DRIVER={SQL Server};SERVER=PISRV01;DATABASE=PIWORLD;Trusted_Connection=Yes;'

# Connect to "Distribution Network Lab" model in AF through the PI SQL Client.
SQL_connection = ado.connect(con_string)

# Create a cursor object to access the data serer for the "Distribution Network Lab" database in AF.
SQL_cursor = SQL_connection.cursor()

# Create linear regression object from the "sklearn" package we imported earlier.
LinReg = LinearRegression()

# Looping through the trasformer list, Perform linear regression on each transformer.
for transformer in allTransformers:
    
    # Create dataframe for one transformer.
    transformerData = modellingData.loc[transformer,:]
    
    # Perform linear regression fit
    LinReg.fit(transformerData[["Wh Load-7d","Wh Load-14d","Temperature","Humidity"]],transformerData["Wh Load"])
    
    # Update asset ID value with the name of this transformer.
    asset_id = "'"+transformer+"'"
    
    # Print equation.
    print(transformer, "Eq:\n", LinReg.coef_[0], "*","'Wh Delivered Load - 7d' + ",LinReg.coef_[1], "*","'Wh Delivered Load - 14d' + ",
    LinReg.coef_[2], "*", "'Ambient Temperature' +", LinReg.coef_[3], "*","'Relative Humidity' +(",LinReg.intercept_, ")" )    
    
    ## INSERT and UPDATE queries to load table for the first time or update an existing one.
    # Construct query to add this transfomer's model coefficients.
    insert_query = f'INSERT [Predictive Equations] ([Asset ID], Coefficient_0, Coefficient_1, Coefficient_2, Coefficient_3, Coefficient_4, Intercept) values({asset_id}, {LinReg.coef_[0]}, {LinReg.coef_[1]}, {LinReg.coef_[2]}, {LinReg.coef_[3]}, null, {LinReg.intercept_})'

    update_query = f'UPDATE [Predictive Equations] SET Coefficient_0={LinReg.coef_[0]}, Coefficient_1={LinReg.coef_[1]},Coefficient_2={LinReg.coef_[2]},Coefficient_3={LinReg.coef_[3]},Intercept={LinReg.intercept_} WHERE [Asset ID]= {asset_id}'
    
    # Insert this record into the "Predictive Equations" table.
    SQL_cursor.execute(update_query)
    
# Commit the queries to write the data into SQL.
SQL_connection.commit()

# Close database connections.
SQL_connection.close()